In [18]:
'''
!pip -q install -U "transformers>=4.44.0" "datasets>=2.19.0" "evaluate>=0.4.0" \
                 "accelerate>=0.30.0" "scikit-learn>=1.5.0" \
                 "peft>=0.12.0" "bitsandbytes>=0.43.0" || True

import torch, transformers, datasets, evaluate, sklearn
print("torch        :", torch.__version__)
print("transformers :", transformers.__version__)
print("datasets     :", datasets.__version__)
print("evaluate     :", evaluate.__version__)
print("sklearn      :", sklearn.__version__)
print("CUDA available:", torch.cuda.is_available())
'''


'\n!pip -q install -U "transformers>=4.44.0" "datasets>=2.19.0" "evaluate>=0.4.0"                  "accelerate>=0.30.0" "scikit-learn>=1.5.0"                  "peft>=0.12.0" "bitsandbytes>=0.43.0" || True\n\nimport torch, transformers, datasets, evaluate, sklearn\nprint("torch        :", torch.__version__)\nprint("transformers :", transformers.__version__)\nprint("datasets     :", datasets.__version__)\nprint("evaluate     :", evaluate.__version__)\nprint("sklearn      :", sklearn.__version__)\nprint("CUDA available:", torch.cuda.is_available())\n'

imports

In [2]:
'''import os, time, random, math, shutil
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

from sklearn.metrics import classification_report, confusion_matrix
import evaluate
import torch.ao.quantization as aoq

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Global config
MODEL_NAME = "bert-base-uncased"
MAX_LENGTH = 96
NUM_LABELS = 6  # sadness, joy, love, anger, fear, surprise

# Training hyperparameters
BASELINE_EPOCHS = 2
BASELINE_LR     = 2e-5
BASELINE_BS     = 32

QAT_EPOCHS      = 2
QAT_LR          = 1e-5
QAT_BS          = 32
FREEZE_LAYERS   = 8   # lower transformer layers to freeze for QAT

QLORA_RANK      = 16
QLORA_ALPHA     = 32
QLORA_DROPOUT   = 0.05
QLORA_EPOCHS    = 2
QLORA_LR        = 2e-4
QLORA_BS        = 32

WEIGHTS_DIR = Path("weights")
WEIGHTS_DIR.mkdir(exist_ok=True, parents=True)

print("Config ready.")'''


Device: cuda
Config ready.


 Load dair-ai/emotion + tokenize

In [3]:
'''
# Load split version: train / validation / test
ds = load_dataset("dair-ai/emotion")
print(ds)

label_list = ds["train"].features["label"].names
id2label = {i: name for i, name in enumerate(label_list)}
label2id = {name: i for i, name in enumerate(label_list)}
print("Labels:", label_list)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess_fn(examples):
    enc = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=MAX_LENGTH,
    )
    enc["labels"] = examples["label"]
    return enc

ds_enc = ds.map(preprocess_fn, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer)

print("Encoded splits:", {k: len(v) for k, v in ds_enc.items()})
'''


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Encoded splits: {'train': 16000, 'validation': 2000, 'test': 2000}


 Metrics & helper

In [4]:
'''
metric_f1  = evaluate.load("f1")
metric_acc = evaluate.load("accuracy")

def compute_macro_f1_acc_from_preds(preds, labels):
    macro_f1 = metric_f1.compute(
        predictions=preds,
        references=labels,
        average="macro"
    )["f1"]
    acc = metric_acc.compute(
        predictions=preds,
        references=labels,
    )["accuracy"]
    return macro_f1, acc

def trainer_compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "macro_f1": metric_f1.compute(predictions=preds, references=p.label_ids, average="macro")["f1"],
        "accuracy": metric_acc.compute(predictions=preds, references=p.label_ids)["accuracy"],
    }

def model_size_mb_from_state_dict(model: torch.nn.Module) -> float:
    tmp = "tmp_state.pt"
    torch.save(model.state_dict(), tmp)
    size_mb = os.path.getsize(tmp) / (1024 * 1024)
    os.remove(tmp)
    return size_mb
    '''


 Baseline Fine-Tuning (FP32/FP16) training part

In [5]:
'''
import inspect
from transformers import TrainingArguments

def make_training_args(**kwargs):
    """
    Version-agnostic wrapper around TrainingArguments.

    - Filters out unsupported kwargs for older transformers.
    - Maps per_device_* -> per_gpu_* if needed.
    - Maps evaluation_strategy -> eval_strategy if needed.
    - If we cannot set an eval strategy but load_best_model_at_end=True,
      we force load_best_model_at_end=False to avoid the ValueError.
    """
    sig = inspect.signature(TrainingArguments.__init__)
    params = sig.parameters

    # --- 1) per_device_* -> per_gpu_* mapping for very old versions ---
    if "per_device_train_batch_size" not in params and "per_gpu_train_batch_size" in params:
        if "per_device_train_batch_size" in kwargs:
            kwargs["per_gpu_train_batch_size"] = kwargs.pop("per_device_train_batch_size")
    if "per_device_eval_batch_size" not in params and "per_gpu_eval_batch_size" in params:
        if "per_device_eval_batch_size" in kwargs:
            kwargs["per_gpu_eval_batch_size"] = kwargs.pop("per_device_eval_batch_size")

    # --- 2) evaluation_strategy -> eval_strategy mapping if needed ---
    if "evaluation_strategy" in kwargs:
        if "evaluation_strategy" not in params and "eval_strategy" in params:
            # map to old name
            kwargs["eval_strategy"] = kwargs.pop("evaluation_strategy")
        elif "evaluation_strategy" not in params and "eval_strategy" not in params:
            # can't set any eval strategy at all, drop it
            print(f"[make_training_args] Dropping unsupported arg: evaluation_strategy={kwargs['evaluation_strategy']!r}")
            kwargs.pop("evaluation_strategy")

    # Build filtered kwargs (drop anything else unsupported)
    filtered = {}
    for k, v in kwargs.items():
        if k in params:
            filtered[k] = v
        else:
            print(f"[make_training_args] Dropping unsupported TrainingArguments arg: {k}={v!r}")

    # --- 3) If we couldn't set any eval strategy but load_best_model_at_end=True, turn it off ---
    # In older versions: attribute name is eval_strategy internally.
    has_eval_arg = ("evaluation_strategy" in params) or ("eval_strategy" in params)
    if not has_eval_arg and filtered.get("load_best_model_at_end", False):
        print("[make_training_args] WARNING: eval strategy cannot be set on this transformers version, "
              "forcing load_best_model_at_end=False to avoid ValueError.")
        filtered["load_best_model_at_end"] = False

    return TrainingArguments(**filtered)


classification

In [6]:
'''
baseline_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
)

baseline_args = make_training_args(
    output_dir="out_baseline",
    per_device_train_batch_size=BASELINE_BS,
    per_device_eval_batch_size=BASELINE_BS,
    learning_rate=BASELINE_LR,
    num_train_epochs=BASELINE_EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=torch.cuda.is_available(),
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    seed=SEED,
    report_to="none",
)

baseline_trainer = Trainer(
    model=baseline_model,
    args=baseline_args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=trainer_compute_metrics,
)

print("Starting baseline training...")
baseline_trainer.train()
print("Baseline training done.")

baseline_save_dir = WEIGHTS_DIR / "baseline_bert"
baseline_trainer.save_model(str(baseline_save_dir))
tokenizer.save_pretrained(str(baseline_save_dir))

print("Saved baseline model to:", baseline_save_dir)
print("Approx. baseline size (MB):", model_size_mb_from_state_dict(baseline_model))
'''


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1673563199.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  baseline_trainer = Trainer(


Starting baseline training...


Epoch,Training Loss,Validation Loss,Macro F1,Accuracy
1,0.248700,0.220817,0.894588,0.918500
2,0.140900,0.160406,0.905218,0.932000


Baseline training done.
Saved baseline model to: weights/baseline_bert
Approx. baseline size (MB): 417.7378034591675


 QAT Training (freeze some layers, simulate quantization)

In [7]:
'''
from torch import nn

# Path to baseline fine-tuned model
BASELINE_DIR = WEIGHTS_DIR / "baseline_bert"

print("Loading BASELINE fine-tuned model as starting point for QAT...")
qat_model = AutoModelForSequenceClassification.from_pretrained(
    str(BASELINE_DIR),
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
)

# Ensuring all weights are float32
qat_model = qat_model.to(torch.float32)

# Move to CPU explicitly for QAT
qat_model = qat_model.to("cpu")

# Freeze lower transformer layers to keep runtime within limits
encoder_layers = qat_model.bert.encoder.layer
for layer in encoder_layers[:FREEZE_LAYERS]:
    for param in layer.parameters():
        param.requires_grad = False

# Leave LayerNorm & embeddings in higher precision (no quantization)
for name, module in qat_model.named_modules():
    if isinstance(module, (nn.LayerNorm, nn.Embedding)):
        module.qconfig = None

# Set default QAT config (int8 fake quantization)
qat_model.qconfig = aoq.get_default_qat_qconfig("fbgemm")


qat_model.train()

print("Preparing model for QAT (CPU, float32) from BASELINE...")
qat_prepared = aoq.prepare_qat(qat_model)
print("QAT-ready model created.")

# 🔹
QAT_TRAIN_SUBSET = 4000  # using batch of 4000
qat_train_dataset = ds_enc["train"].select(range(QAT_TRAIN_SUBSET))
print(f"QAT train subset size: {len(qat_train_dataset)} examples")

# 🔹 Build TrainingArguments
qat_args = make_training_args(
    output_dir="out_qat",
    per_device_train_batch_size=QAT_BS,
    per_device_eval_batch_size=QAT_BS,
    learning_rate=QAT_LR,
    num_train_epochs=2,           # 2 epochs
    evaluation_strategy="no",
    save_strategy="no",
    fp16=False,                   # QAT must stay in float32
    logging_steps=200,
    load_best_model_at_end=False,
    seed=SEED,
    report_to="none",
    no_cuda=True,                 # run entirely on CPU
    save_safetensors=False,

qat_trainer = Trainer(
    model=qat_prepared,
    args=qat_args,
    train_dataset=qat_train_dataset,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,         # metrics will be computed later in inference notebook
)

print("Starting QAT training (CPU, fake quant, float32, SUBSET) FROM BASELINE...")
qat_trainer.train()
print("QAT training done.")

# Now we manually save the final QAT model WITHOUT safetensors
qat_save_dir = WEIGHTS_DIR / "qat_bert"
qat_save_dir.mkdir(parents=True, exist_ok=True)

qat_trainer.model.save_pretrained(str(qat_save_dir), safe_serialization=False)
tokenizer.save_pretrained(str(qat_save_dir))

print("Saved QAT (fake-quant) model to:", qat_save_dir)
print("Approx. QAT model size (MB):", model_size_mb_from_state_dict(qat_trainer.model))
'''

Loading BASELINE fine-tuned model as starting point for QAT...
Preparing model for QAT (CPU, float32) from BASELINE...


/tmp/ipython-input-4058111678.py:38: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  qat_prepared = aoq.prepare_qat(qat_model)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
 

QAT-ready model created.
QAT train subset size: 4000 examples
Starting QAT training (CPU, fake quant, float32, SUBSET) FROM BASELINE...


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/tmp/ipython-input-4058111678.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  qat_trainer = Trainer(


Step,Training Loss
200,0.248800


QAT training done.
Saved QAT (fake-quant) model to: weights/qat_bert
Approx. QAT model size (MB): 419.39062118530273


QLoRA Training (4-bit + LoRA adapters)

In [8]:
'''
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

assert torch.cuda.is_available(), "QLoRA requires GPU with bitsandbytes (4-bit)."

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print("Loading 4-bit BERT model for QLoRA...")
base_4bit = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
    quantization_config=bnb_config,
    device_map="auto",
)

# Prepare model for k-bit (QLoRA)
base_4bit = prepare_model_for_kbit_training(base_4bit)

# Lora config
lora_config = LoraConfig(
    r=QLORA_RANK,
    lora_alpha=QLORA_ALPHA,
    lora_dropout=QLORA_DROPOUT,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "query", "key", "value",
        "dense",
    ],
)

qlora_model = get_peft_model(base_4bit, lora_config)
qlora_model.print_trainable_parameters()

# 2 EPOCHS
qlora_args = make_training_args(
    output_dir="out_qlora",
    per_device_train_batch_size=QLORA_BS,
    per_device_eval_batch_size=QLORA_BS,
    learning_rate=QLORA_LR,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,                       # FP16 is OK for QLoRA
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    seed=SEED,
    report_to="none",
)

qlora_trainer = Trainer(
    model=qlora_model,
    args=qlora_args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=trainer_compute_metrics,
)

print("🚀 Starting QLoRA training for 2 epochs...")
qlora_trainer.train()
print("🎉 QLoRA training finished.")

# Save adapters only (not full model)
qlora_save_dir = WEIGHTS_DIR / "qlora_adapters"
qlora_trainer.model.save_pretrained(str(qlora_save_dir))

print("Saved QLoRA adapters to:", qlora_save_dir)
'''

Loading 4-bit BERT model for QLoRA...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,683,398 || all params: 112,170,252 || trainable%: 2.3923
🚀 Starting QLoRA training for 2 epochs...


/tmp/ipython-input-2184146870.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  qlora_trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Macro F1,Accuracy
1,0.277900,0.225777,0.898226,0.920000
2,0.153500,0.178117,0.908184,0.931000


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


🎉 QLoRA training finished.
Saved QLoRA adapters to: weights/qlora_adapters


 Zipping weights so we can download & upload to GitHub repo

In [9]:
''''
import zipfile

def zip_dir(src_dir: Path, zip_name: str):
    with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(src_dir):
            for f in files:
                full_path = os.path.join(root, f)
                rel_path = os.path.relpath(full_path, src_dir.parent)
                zf.write(full_path, rel_path)

for subdir in ["baseline_bert", "qat_bert", "qlora_adapters"]:
    path = WEIGHTS_DIR / subdir
    if path.exists():
        zip_name = f"{subdir}.zip"
        zip_dir(path, zip_name)
        print("Created:", zip_name)
    else:
        print("Missing, not zipped:", path)
'''

Created: baseline_bert.zip
Created: qat_bert.zip
Created: qlora_adapters.zip


inference part

In [10]:
!pip -q install -U "transformers>=4.44.0" "datasets>=2.19.0" "evaluate>=0.4.0" \
                 "accelerate>=0.30.0" "scikit-learn>=1.5.0" \
                 "peft>=0.12.0" "bitsandbytes>=0.43.0" || True

import os, time, random, shutil
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
)
from peft import PeftModel

from sklearn.metrics import classification_report, confusion_matrix
import evaluate
import torch.ao.quantization as aoq

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

MODEL_NAME = "bert-base-uncased"
MAX_LENGTH = 96
NUM_LABELS = 6


Device: cuda


In [11]:
# Dataset
ds = load_dataset("dair-ai/emotion")
label_list = ds["train"].features["label"].names
id2label = {i: name for i, name in enumerate(label_list)}
label2id = {name: i for i, name in enumerate(label_list)}
print("Labels:", label_list)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess_fn(examples):
    enc = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=MAX_LENGTH,
    )
    enc["labels"] = examples["label"]
    return enc

ds_enc = ds.map(preprocess_fn, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer)

print("Encoded splits:", {k: len(v) for k, v in ds_enc.items()})

# Metrics & helpers
metric_f1  = evaluate.load("f1")
metric_acc = evaluate.load("accuracy")

def compute_macro_f1_acc(y_true, y_pred):
    macro_f1 = metric_f1.compute(
        predictions=y_pred,
        references=y_true,
        average="macro"
    )["f1"]
    acc = metric_acc.compute(
        predictions=y_pred,
        references=y_true
    )["accuracy"]
    return macro_f1, acc

def print_report_and_confusion(y_true, y_pred, title="Model"):
    print(f"\n=== {title} ===")
    print(classification_report(y_true, y_pred, target_names=label_list, digits=4))
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion matrix (rows=true, cols=pred):\n", cm)

def model_size_mb_from_state_dict(model: torch.nn.Module) -> float:
    tmp = "tmp_state.pt"
    torch.save(model.state_dict(), tmp)
    size_mb = os.path.getsize(tmp) / (1024 * 1024)
    os.remove(tmp)
    return size_mb

@torch.inference_mode()
def eval_model(model, dataset, collator, device=device, batch_size=32):
    model.eval()
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)
    all_preds, all_labels = [], []
    for batch in loader:
        labels = batch["labels"].numpy().tolist()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = outputs.logits.argmax(dim=-1).detach().cpu().numpy().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels)
    return np.array(all_labels), np.array(all_preds)

@torch.inference_mode()
def latency_ms_per_example(model, dataset, collator, device=device, batch_size=32, n_batches=30):
    model.eval()
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)
    it = iter(loader)

    # warmup
    for _ in range(2):
        try:
            batch = next(it)
        except StopIteration:
            it = iter(loader)
            batch = next(it)
        batch = {k: v.to(device) for k, v in batch.items()}
        _ = model(**batch)

    times = []
    total_examples = 0
    it = iter(loader)
    for i in range(min(n_batches, len(loader))):
        try:
            batch = next(it)
        except StopIteration:
            break
        bs = batch["labels"].shape[0]
        total_examples += bs
        batch = {k: v.to(device) for k, v in batch.items()}
        if device.type == "cuda":
            torch.cuda.synchronize()
        t0 = time.perf_counter()
        _ = model(**batch)
        if device.type == "cuda":
            torch.cuda.synchronize()
        t1 = time.perf_counter()
        times.append((t1 - t0) * 1000.0)

    if not times or total_examples == 0:
        return 0.0
    avg_ms_per_batch = float(np.mean(times))
    return avg_ms_per_batch / batch_size


Labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Encoded splits: {'train': 16000, 'validation': 2000, 'test': 2000}


In [12]:
# my repo
REPO_URL = "https://github.com/Vishalbharti29/ADVNLP_ASSIGNMENT.git"

repo_dir = Path("repo")
if repo_dir.exists():
    shutil.rmtree(repo_dir)

print("Cloning:", REPO_URL)
!git clone {REPO_URL} repo

BASELINE_DIR = repo_dir / "weights" / "baseline_bert"
QAT_DIR      = repo_dir / "weights" / "qat_bert"
QLORA_DIR    = repo_dir / "weights" / "qlora_adapters"

print("\nCheck expected weight folders:")
print("Baseline:", BASELINE_DIR.exists())
print("QAT     :", QAT_DIR.exists())
print("QLoRA   :", QLORA_DIR.exists())

assert BASELINE_DIR.exists(), "Baseline weights missing at repo/weights/baseline_bert/"


Cloning: https://github.com/Vishalbharti29/ADVNLP_ASSIGNMENT.git
Cloning into 'repo'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 34 (delta 3), reused 31 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 318.10 KiB | 4.54 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Filtering content: 100% (9/9), 848.20 MiB | 8.65 MiB/s, done.

Check expected weight folders:
Baseline: True
QAT     : True
QLoRA   : True


baseline

In [13]:
print("Loading baseline model from:", BASELINE_DIR)
baseline_model = AutoModelForSequenceClassification.from_pretrained(
    str(BASELINE_DIR),
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
).to(device)

y_true_base, y_pred_base = eval_model(
    baseline_model, ds_enc["test"], data_collator, device=device, batch_size=32
)
base_macro_f1, base_acc = compute_macro_f1_acc(y_true_base, y_pred_base)

print_report_and_confusion(y_true_base, y_pred_base, title="Baseline (FP32/FP16) – Test")

base_size_mb = model_size_mb_from_state_dict(baseline_model)
base_latency_ms = latency_ms_per_example(
    baseline_model, ds_enc["test"], data_collator, device=device, batch_size=32, n_batches=30
)

print(f"\n[Baseline] Macro F1: {base_macro_f1:.4f}")
print(f"[Baseline] Accuracy: {base_acc:.4f}")
print(f"[Baseline] Model size (MB): {base_size_mb:.2f}")
print(f"[Baseline] Latency (ms/example, batch=32): {base_latency_ms:.4f}")


Loading baseline model from: repo/weights/baseline_bert

=== Baseline (FP32/FP16) – Test ===
              precision    recall  f1-score   support

     sadness     0.9590    0.9673    0.9632       581
         joy     0.9575    0.9396    0.9484       695
        love     0.8081    0.8742    0.8399       159
       anger     0.9231    0.9164    0.9197       275
        fear     0.8761    0.9152    0.8952       224
    surprise     0.8302    0.6667    0.7395        66

    accuracy                         0.9275      2000
   macro avg     0.8923    0.8799    0.8843      2000
weighted avg     0.9280    0.9275    0.9273      2000

Confusion matrix (rows=true, cols=pred):
 [[562   4   0  10   5   0]
 [  2 653  33   3   0   4]
 [  1  19 139   0   0   0]
 [ 12   2   0 252   9   0]
 [  6   0   0   8 205   5]
 [  3   4   0   0  15  44]]

[Baseline] Macro F1: 0.8843
[Baseline] Accuracy: 0.9275
[Baseline] Model size (MB): 417.74
[Baseline] Latency (ms/example, batch=32): 2.6477


ptq

In [14]:
print("Creating PTQ model (dynamic int8) from baseline (on CPU)...")

baseline_cpu = AutoModelForSequenceClassification.from_pretrained(
    str(BASELINE_DIR),
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
).to("cpu").eval()

ptq_model = aoq.quantize_dynamic(
    baseline_cpu,
    {torch.nn.Linear},
    dtype=torch.qint8
).eval()

@torch.inference_mode()
def eval_model_cpu(model, dataset, collator, batch_size=64):
    model.eval()
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)
    all_preds, all_labels = [], []
    for batch in loader:
        labels = batch["labels"].numpy().tolist()
        batch_cpu = {k: v for k, v in batch.items()}
        out = model(**batch_cpu)
        preds = out.logits.argmax(dim=-1).detach().cpu().numpy().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels)
    return np.array(all_labels), np.array(all_preds)

y_true_ptq, y_pred_ptq = eval_model_cpu(ptq_model, ds_enc["test"], data_collator, batch_size=64)
ptq_macro_f1, ptq_acc = compute_macro_f1_acc(y_true_ptq, y_pred_ptq)

print_report_and_confusion(y_true_ptq, y_pred_ptq, title="PTQ (dynamic int8) – Test")

ptq_size_mb = model_size_mb_from_state_dict(ptq_model)
ptq_latency_ms = latency_ms_per_example(
    ptq_model, ds_enc["test"], data_collator,
    device=torch.device("cpu"),
    batch_size=32,
    n_batches=30,
)

print(f"\n[PTQ] Macro F1: {ptq_macro_f1:.4f}")
print(f"[PTQ] Accuracy: {ptq_acc:.4f}")
print(f"[PTQ] Model size (MB, int8 state_dict): {ptq_size_mb:.2f}")
print(f"[PTQ] Latency (CPU, ms/example, batch=32): {ptq_latency_ms:.4f}")

print("\nPTQ details:")
print("- Quantized layers: torch.nn.Linear")
print("- Weights: int8 (qint8), activations: dynamically quantized")
print("- Embeddings / LayerNorm kept in float32.")


Creating PTQ model (dynamic int8) from baseline (on CPU)...


/tmp/ipython-input-3322741137.py:10: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  ptq_model = aoq.quantize_dynamic(



=== PTQ (dynamic int8) – Test ===
              precision    recall  f1-score   support

     sadness     0.9668    0.9518    0.9592       581
         joy     0.9245    0.9683    0.9459       695
        love     0.8921    0.7799    0.8322       159
       anger     0.9181    0.9382    0.9281       275
        fear     0.8831    0.9107    0.8967       224
    surprise     0.8571    0.6364    0.7304        66

    accuracy                         0.9270      2000
   macro avg     0.9070    0.8642    0.8821      2000
weighted avg     0.9265    0.9270    0.9257      2000

Confusion matrix (rows=true, cols=pred):
 [[553  10   0  11   7   0]
 [  3 673  15   4   0   0]
 [  1  34 124   0   0   0]
 [  7   3   0 258   7   0]
 [  5   0   0   8 204   7]
 [  3   8   0   0  13  42]]

[PTQ] Macro F1: 0.8821
[PTQ] Accuracy: 0.9270
[PTQ] Model size (MB, int8 state_dict): 173.09
[PTQ] Latency (CPU, ms/example, batch=32): 65.8240

PTQ details:
- Quantized layers: torch.nn.Linear
- Weights: int8 (qint8

qat

In [15]:
qat_macro_f1 = qat_acc = qat_size_mb = qat_latency_ms = None
y_true_qat = y_pred_qat = None

if not QAT_DIR.exists():
    print("QAT weights not found at:", QAT_DIR)
    print("-> Skipping QAT evaluation.")
else:
    print("Loading QAT model from:", QAT_DIR)
    qat_model = AutoModelForSequenceClassification.from_pretrained(
        str(QAT_DIR),
        num_labels=NUM_LABELS,
        id2label=id2label,
        label2id=label2id,
    ).to(device).eval()

    y_true_qat, y_pred_qat = eval_model(
        qat_model, ds_enc["test"], data_collator, device=device, batch_size=32
    )
    qat_macro_f1, qat_acc = compute_macro_f1_acc(y_true_qat, y_pred_qat)

    print_report_and_confusion(y_true_qat, y_pred_qat, title="QAT Model – Test")

    qat_size_mb = model_size_mb_from_state_dict(qat_model)
    qat_latency_ms = latency_ms_per_example(
        qat_model, ds_enc["test"], data_collator, device=device, batch_size=32, n_batches=30
    )

    print(f"\n[QAT] Macro F1: {qat_macro_f1:.4f}")
    print(f"[QAT] Accuracy: {qat_acc:.4f}")
    print(f"[QAT] Model size (MB): {qat_size_mb:.2f}")
    print(f"[QAT] Latency (ms/example, batch=32): {qat_latency_ms:.4f}")


Loading QAT model from: repo/weights/qat_bert


Some weights of the model checkpoint at repo/weights/qat_bert were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.0.attention.output.dense.activation_post_process.activation_post_process.eps', 'bert.encoder.layer.0.attention.output.dense.activation_post_process.activation_post_process.max_val', 'bert.encoder.layer.0.attention.output.dense.activation_post_process.activation_post_process.min_val', 'bert.encoder.layer.0.attention.output.dense.activation_post_process.fake_quant_enabled', 'bert.encoder.layer.0.attention.output.dense.activation_post_process.observer_enabled', 'bert.encoder.layer.0.attention.output.dense.activation_post_process.scale', 'bert.encoder.layer.0.attention.output.dense.activation_post_process.zero_point', 'bert.encoder.layer.0.attention.output.dense.weight_fake_quant.activation_post_process.eps', 'bert.encoder.layer.0.attention.output.dense.weight_fake_quant.activation_post_process.max_val', 'bert.encoder.layer.0.attention.output.den


=== QAT Model – Test ===
              precision    recall  f1-score   support

     sadness     0.9653    0.9587    0.9620       581
         joy     0.9729    0.9281    0.9499       695
        love     0.8156    0.9182    0.8639       159
       anger     0.9085    0.9382    0.9231       275
        fear     0.9300    0.8304    0.8774       224
    surprise     0.6392    0.9394    0.7607        66

    accuracy                         0.9270      2000
   macro avg     0.8719    0.9188    0.8895      2000
weighted avg     0.9335    0.9270    0.9285      2000

Confusion matrix (rows=true, cols=pred):
 [[557   4   0  12   7   1]
 [  4 645  33   5   0   8]
 [  1  12 146   0   0   0]
 [  9   2   0 258   6   0]
 [  4   0   0   8 186  26]
 [  2   0   0   1   1  62]]

[QAT] Macro F1: 0.8895
[QAT] Accuracy: 0.9270
[QAT] Model size (MB): 417.74
[QAT] Latency (ms/example, batch=32): 2.7907


qlora

In [16]:
qlora_macro_f1 = qlora_acc = qlora_size_mb = qlora_latency_ms = None
y_true_qlora = y_pred_qlora = None

if not QLORA_DIR.exists():
    print("QLoRA adapters not found at:", QLORA_DIR)
    print("-> Skipping QLoRA evaluation.")
else:
    print("Loading base BERT for QLoRA from:", MODEL_NAME)
    base_for_qlora = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        id2label=id2label,
        label2id=label2id,
    ).to(device)

    print("Loading QLoRA adapters from:", QLORA_DIR)
    qlora_model = PeftModel.from_pretrained(base_for_qlora, str(QLORA_DIR)).to(device).eval()

    y_true_qlora, y_pred_qlora = eval_model(
        qlora_model, ds_enc["test"], data_collator, device=device, batch_size=32
    )
    qlora_macro_f1, qlora_acc = compute_macro_f1_acc(y_true_qlora, y_pred_qlora)

    print_report_and_confusion(y_true_qlora, y_pred_qlora, title="QLoRA (adapters) – Test")

    # adapter size
    def dir_size_mb(path):
        total = 0
        for root, _, files in os.walk(path):
            for f in files:
                fp = os.path.join(root, f)
                total += os.path.getsize(fp)
        return total / (1024 * 1024)

    adapter_size_mb = dir_size_mb(str(QLORA_DIR))
    base_size_mb_for_qlora = base_size_mb  # reuse baseline size
    qlora_size_mb = base_size_mb_for_qlora + adapter_size_mb

    qlora_latency_ms = latency_ms_per_example(
        qlora_model, ds_enc["test"], data_collator, device=device, batch_size=32, n_batches=30
    )

    print(f"\n[QLoRA] Macro F1: {qlora_macro_f1:.4f}")
    print(f"[QLoRA] Accuracy: {qlora_acc:.4f}")
    print(f"[QLoRA] Adapter size only (MB): {adapter_size_mb:.2f}")
    print(f"[QLoRA] Effective model size (base+adapters, MB): {qlora_size_mb:.2f}")
    print(f"[QLoRA] Latency (ms/example, batch=32): {qlora_latency_ms:.4f}")

    print("\nQLoRA hyperparameters:")
    print("r =", QLORA_RANK, "| alpha =", QLORA_ALPHA, "| dropout =", QLORA_DROPOUT)


Loading base BERT for QLoRA from: bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading QLoRA adapters from: repo/weights/qlora_adapters

=== QLoRA (adapters) – Test ===
              precision    recall  f1-score   support

     sadness     0.9588    0.9621    0.9605       581
         joy     0.9556    0.9295    0.9424       695
        love     0.7898    0.8742    0.8299       159
       anger     0.9225    0.9091    0.9158       275
        fear     0.8644    0.9107    0.8870       224
    surprise     0.7759    0.6818    0.7258        66

    accuracy                         0.9215      2000
   macro avg     0.8778    0.8779    0.8769      2000
weighted avg     0.9227    0.9215    0.9217      2000

Confusion matrix (rows=true, cols=pred):
 [[559   4   1  11   6   0]
 [  4 646  36   3   1   5]
 [  0  20 139   0   0   0]
 [ 13   3   0 250   9   0]
 [  5   0   0   7 204   8]
 [  2   3   0   0  16  45]]

[QLoRA] Macro F1: 0.8769
[QLoRA] Accuracy: 0.9215
[QLoRA] Adapter size only (MB): 10.26
[QLoRA] Effective model size (base+adapters, MB): 428.00
[QLoRA] Latency 

table

In [17]:
rows = []

rows.append({
    "Model": "Baseline",
    "Macro F1": base_macro_f1,
    "Accuracy": base_acc,
    "Size (MB)": base_size_mb,
    "Latency (ms/example)": base_latency_ms,
    "Notes": "Full fine-tune FP32/FP16",
})

rows.append({
    "Model": "PTQ (dynamic int8)",
    "Macro F1": ptq_macro_f1,
    "Accuracy": ptq_acc,
    "Size (MB)": ptq_size_mb,
    "Latency (ms/example)": ptq_latency_ms,
    "Notes": "Dynamic int8 Linear layers (CPU)",
})

rows.append({
    "Model": "QAT",
    "Macro F1": float(qat_macro_f1) if qat_macro_f1 is not None else None,
    "Accuracy": float(qat_acc) if qat_acc is not None else None,
    "Size (MB)": float(qat_size_mb) if qat_size_mb is not None else None,
    "Latency (ms/example)": float(qat_latency_ms) if qat_latency_ms is not None else None,
    "Notes": "Fake-quant QAT model",
})

rows.append({
    "Model": "QLoRA",
    "Macro F1": float(qlora_macro_f1) if qlora_macro_f1 is not None else None,
    "Accuracy": float(qlora_acc) if qlora_acc is not None else None,
    "Size (MB)": float(qlora_size_mb) if qlora_size_mb is not None else None,
    "Latency (ms/example)": float(qlora_latency_ms) if qlora_latency_ms is not None else None,
    "Notes": "4-bit base + low-rank adapters",
})

df = pd.DataFrame(rows)
print("\n=== Final Comparison Table (Baseline vs PTQ vs QAT vs QLoRA) ===")
print(df.to_string(index=False))

df.to_csv("results_summary.csv", index=False)
print("\nSaved results_summary.csv")



=== Final Comparison Table (Baseline vs PTQ vs QAT vs QLoRA) ===
             Model  Macro F1  Accuracy  Size (MB)  Latency (ms/example)                            Notes
          Baseline  0.884312    0.9275 417.737803              2.647654         Full fine-tune FP32/FP16
PTQ (dynamic int8)  0.882089    0.9270 173.089793             65.824015 Dynamic int8 Linear layers (CPU)
               QAT  0.889501    0.9270 417.737803              2.790726             Fake-quant QAT model
             QLoRA  0.876871    0.9215 427.999663              3.450257   4-bit base + low-rank adapters

Saved results_summary.csv
